# SAM 2 Lake Detection with Ground Truth Tuning - Google Colab Version

This notebook systematically tunes SAM 2 parameters for glacial lake detection using ground truth data.

**Prerequisites:**
- Upload your images and ground truth mask to Google Drive
- Enable GPU runtime (Runtime → Change runtime type → GPU)

**Workflow:**
1. Mount Google Drive and setup environment
2. Load satellite image (RGB/FCC/NDWI) and ground truth
3. Test different SAM 2 parameter combinations
4. Find optimal parameters using IoU, precision, recall
5. Generate final results with best configuration

## 🔧 Environment Setup

In [ ]:
# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
    print("🔵 Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("🟡 Running locally")

In [ ]:
# Mount Google Drive
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive mounted at /content/drive")
    
    # List Drive contents to help you find your files
    import os
    print("\n📁 Your Google Drive contents:")
    drive_path = "/content/drive/MyDrive"
    if os.path.exists(drive_path):
        for item in os.listdir(drive_path)[:10]:  # Show first 10 items
            print(f"  📂 {item}")
        if len(os.listdir(drive_path)) > 10:
            print(f"  ... and {len(os.listdir(drive_path)) - 10} more items")
    else:
        print("  Drive path not found. Check mounting.")

In [ ]:
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print("✅ GPU is ready for SAM 2!")
else:
    print("⚠️ No GPU detected!")
    print("Go to Runtime → Change runtime type → Hardware accelerator → GPU")
    print("Then restart the runtime.")

In [ ]:
# Install required packages
print("📦 Installing required packages...")

# Core packages
!pip install opencv-python matplotlib scikit-learn pandas
!pip install rasterio geopandas

# SAM 2
!pip install 'git+https://github.com/facebookresearch/sam2.git'

# TorchGeo (optional, for advanced geospatial processing)
!pip install torchgeo

print("✅ Package installation complete!")

In [ ]:
# Download SAM 2 model weights
import os

model_dir = "/content/sam2_models"
os.makedirs(model_dir, exist_ok=True)

model_path = f"{model_dir}/sam2.1_hiera_large.pt"

if not os.path.exists(model_path):
    print("📥 Downloading SAM 2 model (856MB)...")
    !wget -P {model_dir} https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_large.pt
    print("✅ SAM 2 model downloaded!")
else:
    print("✅ SAM 2 model already exists!")

# Also download config files
config_dir = "/content/sam2_configs"
os.makedirs(config_dir, exist_ok=True)

if not os.path.exists(f"{config_dir}/sam2.1_hiera_l.yaml"):
    print("📥 Downloading SAM 2 config files...")
    !wget -P {config_dir} https://raw.githubusercontent.com/facebookresearch/sam2/main/sam2/configs/sam2.1/sam2.1_hiera_l.yaml
    !wget -P {config_dir} https://raw.githubusercontent.com/facebookresearch/sam2/main/sam2/configs/sam2.1/sam2.1_hiera_b+.yaml
    !wget -P {config_dir} https://raw.githubusercontent.com/facebookresearch/sam2/main/sam2/configs/sam2.1/sam2.1_hiera_s.yaml
    print("✅ Config files downloaded!")
else:
    print("✅ Config files already exist!")

## 📚 Import Libraries

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import json
from sklearn.metrics import jaccard_score, precision_score, recall_score
import warnings
warnings.filterwarnings('ignore')

# Geospatial libraries
import rasterio
import geopandas as gpd

# PyTorch and SAM 2
import torch
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
from sam2.sam2_image_predictor import SAM2ImagePredictor

# TorchGeo (if needed)
try:
    import torchgeo
    print(f"TorchGeo version: {torchgeo.__version__}")
except ImportError:
    print("TorchGeo not available (optional)")

print(f"✅ All libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"NumPy version: {np.__version__}")

## ⚙️ Configuration - UPDATE YOUR FILE PATHS

**Important:** Update the paths below to point to your files in Google Drive

In [ ]:
# =============================================================================
# 🚨 UPDATE THESE PATHS TO YOUR GOOGLE DRIVE FILES
# =============================================================================

# Base path to your data in Google Drive
# Example: if your files are in "My Drive/lake_data/", use:
DRIVE_DATA_PATH = "/content/drive/MyDrive/superlakes/"  # UPDATE THIS

# Choose your input image type
IMAGE_TYPE = "RGB"  # Options: "RGB", "FCC", "NDWI" - RGB often works best with SAM 2!

# Image file paths (relative to DRIVE_DATA_PATH)
IMAGE_FILES = {
    "RGB": "2021-09-04_rgb_testclip_sam2.tif",   # True color RGB (often works best!)
    "FCC": "2021-09-04_fcc_testclip.tif",       # False Color Composite
    "NDWI": "2021-09-04_fndwi_clip_sam.tif"     # NDWI single band
}

# Ground truth file (binary mask: 1=lake, 0=not lake)
GROUND_TRUTH_FILE = "lake_mask_testclip.tif"  # UPDATE THIS

# SAM 2 model configuration
SAM2_CONFIG = {
    "checkpoint": "/content/sam2_models/sam2.1_hiera_large.pt",
    "config": "/content/sam2_configs/sam2.1_hiera_l.yaml"
}

# =============================================================================

# Construct full paths
IMAGE_PATH = os.path.join(DRIVE_DATA_PATH, IMAGE_FILES[IMAGE_TYPE])
GROUND_TRUTH_PATH = os.path.join(DRIVE_DATA_PATH, GROUND_TRUTH_FILE)

print(f"🎯 Configuration:")
print(f"   Image type: {IMAGE_TYPE} ← RGB often works best with SAM 2!")
print(f"   Image path: {IMAGE_PATH}")
print(f"   Ground truth: {GROUND_TRUTH_PATH}")
print(f"   SAM 2 model: {SAM2_CONFIG['checkpoint']}")

# Verify files exist
print(f"\n📁 File verification:")
print(f"   Image exists: {'✅' if os.path.exists(IMAGE_PATH) else '❌'}")
print(f"   Ground truth exists: {'✅' if os.path.exists(GROUND_TRUTH_PATH) else '❌'}")
print(f"   SAM 2 model exists: {'✅' if os.path.exists(SAM2_CONFIG['checkpoint']) else '❌'}")

if not os.path.exists(DRIVE_DATA_PATH):
    print(f"\n⚠️ Data directory not found: {DRIVE_DATA_PATH}")
    print("Please update DRIVE_DATA_PATH to your actual Google Drive folder.")

print(f"\n💡 Tip: Try different image types by changing IMAGE_TYPE above!")
print(f"   RGB: Often best for SAM 2 (trained on natural images)")
print(f"   FCC: Good for vegetation/water contrast") 
print(f"   NDWI: Emphasizes water but may confuse SAM 2")

## 🖥️ Device Setup

In [ ]:
# Set up compute device
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"🚀 Using GPU: {torch.cuda.get_device_name(0)}")
    
    # Optimize for GPU
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
        print("✅ TensorFloat-32 optimization enabled")
        
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("🍎 Using Apple Metal Performance Shaders (MPS)")
    print("Note: MPS support is experimental and may give different results")
else:
    device = torch.device("cpu")
    print("💻 Using CPU (will be slower)")

print(f"Device: {device}")

## 🛠️ Utility Functions

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

def show_anns(anns, borders=True):
    """Visualize SAM 2 annotations with random colors"""
    if len(anns) == 0:
        return
    
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:, :, 3] = 0
    
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.5]])
        img[m] = color_mask
        
        if borders:
            import cv2
            contours, _ = cv2.findContours(m.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            contours = [cv2.approxPolyDP(contour, epsilon=0.01, closed=True) for contour in contours]
            cv2.drawContours(img, contours, -1, (0, 0, 1, 0.4), thickness=1)

    ax.imshow(img)

def load_image_data(image_path, image_type):
    """Load and prepare image data based on type"""
    print(f"Loading {image_type} image from: {image_path}")
    
    if image_type == "FCC" or image_type == "RGB":
        # False Color Composite or RGB
        image = Image.open(image_path)
        image = np.array(image.convert("RGB"))
        
    elif image_type == "NDWI":
        # NDWI converted to 3-channel
        with rasterio.open(image_path) as src:
            ndwi = src.read(1)
        
        # Normalize NDWI from [-1, 1] to [0, 255]
        ndwi_normalized = ((ndwi + 1) / 2 * 255).astype(np.uint8)
        
        # Create 3-channel array
        image = np.stack([ndwi_normalized, ndwi_normalized, ndwi_normalized], axis=-1)
    
    else:
        raise ValueError(f"Unknown image type: {image_type}")
    
    return image

def calculate_metrics(ground_truth, predicted_mask):
    """Calculate IoU, precision, recall, F1 score"""
    gt_flat = ground_truth.flatten()
    pred_flat = predicted_mask.flatten()
    
    iou = jaccard_score(gt_flat, pred_flat)
    precision = precision_score(gt_flat, pred_flat, zero_division=0)
    recall = recall_score(gt_flat, pred_flat, zero_division=0)
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'iou': iou,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

print("✅ Utility functions defined")

## 📂 Load Image and Ground Truth Data

In [ ]:
# Load satellite image
try:
    image = load_image_data(IMAGE_PATH, IMAGE_TYPE)
    print(f"✅ Image loaded successfully!")
    print(f"   Shape: {image.shape}")
    print(f"   Data type: {image.dtype}")
    print(f"   Value range: {image.min()} - {image.max()}")
    
    # Visualize the image
    plt.figure(figsize=(12, 12))
    plt.imshow(image)
    plt.title(f'{IMAGE_TYPE} Satellite Image\nShape: {image.shape}')
    plt.axis('off')
    plt.show()
    
    IMAGE_LOADED = True
    
except Exception as e:
    print(f"❌ Error loading image: {e}")
    print(f"Check your image path: {IMAGE_PATH}")
    IMAGE_LOADED = False
    image = None

In [ ]:
# Load ground truth lake mask
try:
    with rasterio.open(GROUND_TRUTH_PATH) as src:
        ground_truth = src.read(1).astype(bool)
    
    print(f"✅ Ground truth loaded successfully!")
    print(f"   Shape: {ground_truth.shape}")
    print(f"   Lake pixels: {ground_truth.sum():,} ({ground_truth.mean()*100:.2f}% of image)")
    
    # Check dimension compatibility
    if IMAGE_LOADED and ground_truth.shape != image.shape[:2]:
        print(f"⚠️ WARNING: Dimension mismatch!")
        print(f"   Image: {image.shape[:2]}")
        print(f"   Ground truth: {ground_truth.shape}")
        print("   Consider resizing or checking coordinate systems.")
    elif IMAGE_LOADED:
        print("✅ Image and ground truth dimensions match!")
    
    # Visualize ground truth overlay
    if IMAGE_LOADED:
        fig, axes = plt.subplots(1, 2, figsize=(20, 10))
        
        # Original image
        axes[0].imshow(image)
        axes[0].set_title(f'{IMAGE_TYPE} Image')
        axes[0].axis('off')
        
        # Image with ground truth overlay
        axes[1].imshow(image)
        axes[1].imshow(ground_truth, alpha=0.6, cmap='Blues')
        axes[1].set_title(f'Ground Truth Lakes Overlay\n{ground_truth.sum():,} lake pixels')
        axes[1].axis('off')
        
        plt.tight_layout()
        plt.show()
    
    GROUND_TRUTH_LOADED = True
    
except Exception as e:
    print(f"❌ Error loading ground truth: {e}")
    print(f"Check your ground truth path: {GROUND_TRUTH_PATH}")
    print("Will proceed without parameter tuning.")
    GROUND_TRUTH_LOADED = False
    ground_truth = None

## 🤖 Initialize SAM 2 Model

In [ ]:
# Initialize SAM 2 model
if IMAGE_LOADED:
    try:
        print("🤖 Loading SAM 2 model...")
        
        # Use relative config path like in the example notebooks
        # This works because build_sam2 looks for configs relative to the sam2 package
        model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"
        
        print(f"   Config: {model_cfg} (relative path)")
        print(f"   Checkpoint: {SAM2_CONFIG['checkpoint']}")
        
        # Build SAM 2 model using relative config path (like example notebooks)
        sam2_model = build_sam2(
            model_cfg,
            SAM2_CONFIG['checkpoint'], 
            device=device, 
            apply_postprocessing=False
        )
        
        print("✅ SAM 2 model loaded successfully!")
        print(f"   Model device: {next(sam2_model.parameters()).device}")
        
        SAM2_LOADED = True
        
    except Exception as e:
        print(f"❌ Error loading SAM 2: {e}")
        print("Make sure SAM 2 was installed correctly with: pip install 'git+https://github.com/facebookresearch/sam2.git'")
        SAM2_LOADED = False
        sam2_model = None
else:
    print("⏸️ Skipping SAM 2 initialization (no image loaded)")
    SAM2_LOADED = False
    sam2_model = None

## 🧪 Quick Test with Default Parameters

In [ ]:
# Quick test with default SAM 2 parameters
if SAM2_LOADED:
    print("🧪 Running quick test with default parameters...")
    
    # Create default mask generator
    default_generator = SAM2AutomaticMaskGenerator(sam2_model)
    
    # Generate masks
    print("   Generating masks... (this may take a minute)")
    default_masks = default_generator.generate(image)
    
    print(f"✅ Default configuration generated {len(default_masks)} masks")
    
    # Quick evaluation if ground truth is available
    if GROUND_TRUTH_LOADED:
        # Combine masks
        combined_mask = np.zeros_like(ground_truth, dtype=bool)
        for mask in default_masks:
            combined_mask |= mask['segmentation']
        
        # Calculate metrics
        default_metrics = calculate_metrics(ground_truth, combined_mask)
        print(f"   Default IoU: {default_metrics['iou']:.3f}")
        print(f"   Default Precision: {default_metrics['precision']:.3f}")
        print(f"   Default Recall: {default_metrics['recall']:.3f}")
        print(f"   Default F1: {default_metrics['f1']:.3f}")
    
    # Visualize result
    plt.figure(figsize=(16, 16))
    plt.imshow(image)
    show_anns(default_masks)
    plt.title(f'Default SAM 2 Result\n{len(default_masks)} masks detected')
    plt.axis('off')
    plt.show()
    
    DEFAULT_TEST_DONE = True
    
else:
    print("⏸️ Skipping default test (SAM 2 not loaded)")
    DEFAULT_TEST_DONE = False
    default_masks = []
    default_metrics = {}

In [ ]:
# Detailed comparison: Default SAM 2 vs Ground Truth
if DEFAULT_TEST_DONE and GROUND_TRUTH_LOADED:
    print("📊 Creating detailed comparison of default SAM 2 vs ground truth...")
    
    # Combine default masks into binary array
    default_combined = np.zeros_like(ground_truth, dtype=bool)
    for mask in default_masks:
        default_combined |= mask['segmentation']
    
    # Create side-by-side comparison
    fig, axes = plt.subplots(1, 2, figsize=(24, 12))
    
    # Left panel: Ground Truth
    axes[0].imshow(image)
    axes[0].imshow(ground_truth, alpha=0.7, cmap='Blues')
    axes[0].set_title(f'Ground Truth Lakes\n{ground_truth.sum():,} pixels ({ground_truth.mean()*100:.1f}% of image)', 
                     fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    # Right panel: Default SAM 2 Result
    axes[1].imshow(image)
    axes[1].imshow(default_combined, alpha=0.7, cmap='Reds')
    axes[1].set_title(f'Default SAM 2 Detection\n{default_combined.sum():,} pixels ({default_combined.mean()*100:.1f}% of image)', 
                     fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    # Add performance metrics as text
    fig.suptitle(f'Default SAM 2 Performance: IoU={default_metrics["iou"]:.3f}, Precision={default_metrics["precision"]:.3f}, Recall={default_metrics["recall"]:.3f}, F1={default_metrics["f1"]:.3f}', 
                 fontsize=16, fontweight='bold', y=0.02)
    
    plt.tight_layout()
    plt.show()
    
    # Create detailed pixel-level analysis
    print("\n🔍 Detailed pixel-level analysis:")
    
    # Calculate pixel categories
    true_positive = (ground_truth & default_combined).sum()
    false_positive = (~ground_truth & default_combined).sum()
    false_negative = (ground_truth & ~default_combined).sum()
    true_negative = (~ground_truth & ~default_combined).sum()
    
    total_pixels = ground_truth.size
    
    print(f"   ✅ True Positives (correctly detected lakes): {true_positive:,} pixels")
    print(f"   ❌ False Positives (incorrectly detected as lakes): {false_positive:,} pixels")
    print(f"   ⭕ False Negatives (missed lake pixels): {false_negative:,} pixels")
    print(f"   ✅ True Negatives (correctly identified non-lakes): {true_negative:,} pixels")
    
    print(f"\n📈 Analysis:")
    if false_positive > false_negative:
        print(f"   🔴 SAM 2 is over-detecting (too many false positives)")
        print(f"   💡 Suggestion: Increase quality thresholds or minimum area")
    elif false_negative > false_positive:
        print(f"   🔵 SAM 2 is under-detecting (missing lakes)")
        print(f"   💡 Suggestion: Decrease thresholds or increase sampling density")
    else:
        print(f"   ⚖️ Balanced detection errors")
    
    # Show overlap visualization
    fig, ax = plt.subplots(1, 1, figsize=(16, 12))
    
    # Create color-coded comparison
    comparison = np.zeros((*ground_truth.shape, 3), dtype=np.uint8)
    comparison[ground_truth & default_combined] = [0, 255, 0]        # True positive = Green
    comparison[~ground_truth & default_combined] = [255, 0, 0]       # False positive = Red  
    comparison[ground_truth & ~default_combined] = [0, 0, 255]       # False negative = Blue
    
    ax.imshow(image)
    ax.imshow(comparison, alpha=0.6)
    ax.set_title('Pixel-Level Comparison\n🟢 Correct Detection  🔴 False Positives  🔵 Missed Lakes', 
                fontsize=14, fontweight='bold')
    ax.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n🎯 Summary: Default SAM 2 detects {default_metrics['recall']*100:.1f}% of your lakes but generates {false_positive:,} false positive pixels")
    
else:
    print("⏸️ Skipping detailed comparison (default test or ground truth not available)")

In [ ]:
# Define parameter combinations to test
if SAM2_LOADED and GROUND_TRUTH_LOADED:
    print("🎯 Setting up parameter tuning experiments...")
    
    param_configs = [
        # Baseline: Default configuration for comparison
        {"points_per_side": 32, "pred_iou_thresh": 0.88, "min_mask_region_area": 100, "stability_score_thresh": 0.95, "name": "default"},
        
        # SMALL LAKE FOCUSED CONFIGURATIONS
        # Reduce minimum area to catch small lakes
        {"points_per_side": 32, "pred_iou_thresh": 0.88, "min_mask_region_area": 25, "stability_score_thresh": 0.95, "name": "small_lakes_v1"},
        {"points_per_side": 32, "pred_iou_thresh": 0.88, "min_mask_region_area": 10, "stability_score_thresh": 0.95, "name": "tiny_lakes_v1"},
        {"points_per_side": 32, "pred_iou_thresh": 0.88, "min_mask_region_area": 5, "stability_score_thresh": 0.95, "name": "micro_lakes"},
        
        # Increase sampling density for small features
        {"points_per_side": 48, "pred_iou_thresh": 0.88, "min_mask_region_area": 25, "stability_score_thresh": 0.95, "name": "dense_small_lakes"},
        {"points_per_side": 64, "pred_iou_thresh": 0.88, "min_mask_region_area": 25, "stability_score_thresh": 0.95, "name": "very_dense_small"},
        {"points_per_side": 80, "pred_iou_thresh": 0.88, "min_mask_region_area": 15, "stability_score_thresh": 0.95, "name": "ultra_dense_small"},
        
        # Relax quality thresholds while keeping small area detection
        {"points_per_side": 48, "pred_iou_thresh": 0.8, "min_mask_region_area": 20, "stability_score_thresh": 0.9, "name": "relaxed_small_v1"},
        {"points_per_side": 64, "pred_iou_thresh": 0.75, "min_mask_region_area": 15, "stability_score_thresh": 0.88, "name": "relaxed_small_v2"},
        {"points_per_side": 48, "pred_iou_thresh": 0.82, "min_mask_region_area": 30, "stability_score_thresh": 0.92, "name": "balanced_small"},
        
        # MULTI-SCALE APPROACHES
        # Try to get both small and large lakes
        {"points_per_side": 40, "pred_iou_thresh": 0.85, "min_mask_region_area": 20, "stability_score_thresh": 0.93, "name": "multi_scale_v1"},
        {"points_per_side": 56, "pred_iou_thresh": 0.83, "min_mask_region_area": 35, "stability_score_thresh": 0.91, "name": "multi_scale_v2"},
        
        # AGGRESSIVE DETECTION (may have more false positives but catch more lakes)
        {"points_per_side": 64, "pred_iou_thresh": 0.7, "min_mask_region_area": 10, "stability_score_thresh": 0.85, "name": "aggressive_detection"},
        {"points_per_side": 48, "pred_iou_thresh": 0.75, "min_mask_region_area": 15, "stability_score_thresh": 0.87, "name": "moderate_aggressive"},
        
        # FINE-TUNED VARIANTS (based on default but optimized for small lakes)
        {"points_per_side": 40, "pred_iou_thresh": 0.86, "min_mask_region_area": 20, "stability_score_thresh": 0.94, "name": "default_plus_small"},
        {"points_per_side": 36, "pred_iou_thresh": 0.84, "min_mask_region_area": 40, "stability_score_thresh": 0.93, "name": "slightly_relaxed"},
    ]
    
    print(f"📋 Will test {len(param_configs)} parameter combinations:")
    print("🎯 Focus: Detect small lakes while maintaining good performance on larger ones")
    print("\nConfiguration types:")
    print("  🔵 Default: Baseline for comparison")
    print("  🟢 Small lake focused: Lower min area, higher sampling")
    print("  🟡 Multi-scale: Balance small and large lake detection") 
    print("  🟠 Aggressive: Lower thresholds to catch more lakes")
    print("  🟣 Fine-tuned: Modest improvements over default")
    print()
    
    for i, config in enumerate(param_configs, 1):
        category = "🔵" if "default" in config['name'] else \
                  "🟢" if "small" in config['name'] or "tiny" in config['name'] or "micro" in config['name'] else \
                  "🟡" if "multi" in config['name'] else \
                  "🟠" if "aggressive" in config['name'] else "🟣"
        print(f"   {i:2d}. {category} {config['name']}")
    
    TUNING_READY = True
    
else:
    print("⏸️ Skipping parameter tuning setup")
    if not SAM2_LOADED:
        print("   Reason: SAM 2 not loaded")
    if not GROUND_TRUTH_LOADED:
        print("   Reason: Ground truth not loaded")
    
    TUNING_READY = False
    param_configs = []

In [ ]:
# Run parameter tuning experiments
if TUNING_READY:
    print("🚀 Starting parameter tuning experiments...")
    print(f"This will test {len(param_configs)} configurations.")
    print("Each test may take 1-3 minutes depending on parameters.\n")
    
    results = []
    
    for i, config in enumerate(param_configs):
        config_name = config.pop('name')  # Remove name from config dict
        print(f"Testing {i+1}/{len(param_configs)}: {config_name}")
        print(f"   Parameters: {config}")
        
        try:
            # Create mask generator with current configuration
            mask_generator = SAM2AutomaticMaskGenerator(sam2_model, **config)
            
            # Generate masks
            print("   Generating masks...")
            masks = mask_generator.generate(image)
            
            # Combine masks into binary map
            combined_mask = np.zeros_like(ground_truth, dtype=bool)
            for mask in masks:
                combined_mask |= mask['segmentation']
            
            # Calculate metrics
            metrics = calculate_metrics(ground_truth, combined_mask)
            
            # Store results
            result = {
                'config_name': config_name,
                'config_id': i,
                **config,  # Add all config parameters
                **metrics,  # Add all metrics
                'num_masks': len(masks),
                'predicted_lake_pixels': combined_mask.sum(),
                'true_lake_pixels': ground_truth.sum(),
                'coverage_ratio': combined_mask.sum() / ground_truth.sum() if ground_truth.sum() > 0 else 0
            }
            results.append(result)
            
            print(f"   ✅ IoU: {metrics['iou']:.3f}, Precision: {metrics['precision']:.3f}, Recall: {metrics['recall']:.3f}, F1: {metrics['f1']:.3f}")
            print(f"      Masks: {len(masks)}, Predicted pixels: {combined_mask.sum():,}\n")
            
        except Exception as e:
            print(f"   ❌ Error: {str(e)}\n")
            continue
    
    print(f"✅ Parameter tuning complete! Tested {len(results)} configurations successfully.")
    
    # Convert to DataFrame for analysis
    if results:
        df_results = pd.DataFrame(results)
        TUNING_COMPLETE = True
    else:
        print("❌ No successful configurations")
        TUNING_COMPLETE = False
        df_results = pd.DataFrame()
        
else:
    print("⏸️ Skipping parameter tuning execution")
    TUNING_COMPLETE = False
    results = []
    df_results = pd.DataFrame()

# Analyze and rank results
if TUNING_COMPLETE and not df_results.empty:
    print("📊 Analyzing parameter tuning results...\n")
    
    # Create multiple ranking approaches for small lake detection
    df_results['f1_score'] = df_results['f1']  # For clarity
    df_results['recall_weighted_score'] = 0.6 * df_results['recall'] + 0.4 * df_results['precision']  # Favor recall
    df_results['balanced_score'] = 0.5 * df_results['recall'] + 0.5 * df_results['precision']  # Balanced
    
    print("🏆 RANKING BY DIFFERENT CRITERIA:")
    print("=" * 100)
    
    # 1. Recall-focused (best for catching small lakes)
    print("\n🎯 BEST FOR SMALL LAKE DETECTION (Ranked by Recall):")
    df_recall = df_results.sort_values('recall', ascending=False)
    top_recall = df_recall.head(5)
    for idx, (_, row) in enumerate(top_recall.iterrows(), 1):
        print(f"Rank {idx}: {row['config_name']}")
        print(f"         📊 Recall: {row['recall']:.3f}, Precision: {row['precision']:.3f}, F1: {row['f1']:.3f}")
        print(f"         ⚙️  Points: {row['points_per_side']:2d}, Area: {row['min_mask_region_area']:3d}, IoU: {row['pred_iou_thresh']:.2f}")
        print()
    
    # 2. Balanced approach
    print("⚖️ BEST BALANCED PERFORMANCE (Ranked by F1 Score):")
    df_f1 = df_results.sort_values('f1', ascending=False)
    top_f1 = df_f1.head(5)
    for idx, (_, row) in enumerate(top_f1.iterrows(), 1):
        print(f"Rank {idx}: {row['config_name']}")
        print(f"         📊 F1: {row['f1']:.3f}, Recall: {row['recall']:.3f}, Precision: {row['precision']:.3f}")
        print(f"         ⚙️  Points: {row['points_per_side']:2d}, Area: {row['min_mask_region_area']:3d}, IoU: {row['pred_iou_thresh']:.2f}")
        print()
    
    # 3. Traditional IoU ranking (for comparison)
    print("🔷 TRADITIONAL RANKING (Ranked by IoU):")
    df_iou = df_results.sort_values('iou', ascending=False)
    top_iou = df_iou.head(3)
    for idx, (_, row) in enumerate(top_iou.iterrows(), 1):
        print(f"Rank {idx}: {row['config_name']}")
        print(f"         📊 IoU: {row['iou']:.3f}, Recall: {row['recall']:.3f}, Precision: {row['precision']:.3f}")
        print()
    
    # 4. Compare with default
    default_row = df_results[df_results['config_name'] == 'default']
    if not default_row.empty:
        default_row = default_row.iloc[0]
        print("🔵 DEFAULT CONFIGURATION PERFORMANCE:")
        print(f"     📊 Recall: {default_row['recall']:.3f}, Precision: {default_row['precision']:.3f}, F1: {default_row['f1']:.3f}")
        print(f"     📈 Rank by Recall: #{df_recall[df_recall['config_name'] == 'default'].index[0] + 1} of {len(df_results)}")
        print(f"     📈 Rank by F1: #{df_f1[df_f1['config_name'] == 'default'].index[0] + 1} of {len(df_results)}")
        print()
    
    # Choose best configuration based on recall (small lake detection)
    best_config = df_recall.iloc[0]
    print(f"🎯 RECOMMENDED FOR SMALL LAKE DETECTION:")
    print(f"   Name: {best_config['config_name']}")
    print(f"   points_per_side: {best_config['points_per_side']}")
    print(f"   pred_iou_thresh: {best_config['pred_iou_thresh']}")
    print(f"   min_mask_region_area: {best_config['min_mask_region_area']}")
    print(f"   stability_score_thresh: {best_config['stability_score_thresh']}")
    print(f"   📈 Performance: Recall={best_config['recall']:.3f}, Precision={best_config['precision']:.3f}, F1={best_config['f1']:.3f}")
    
    # Analysis of small vs large lake trade-offs
    print(f"\n🔍 SMALL LAKE DETECTION ANALYSIS:")
    print(f"   💧 Best recall detects {best_config['num_masks']} total segments")
    print(f"   💧 Default detects {default_row['num_masks'] if not default_row.empty else 'N/A'} total segments")
    
    if not default_row.empty:
        recall_improvement = (best_config['recall'] - default_row['recall']) / default_row['recall'] * 100
        precision_change = (best_config['precision'] - default_row['precision']) / default_row['precision'] * 100
        print(f"   📈 Recall improvement: {recall_improvement:+.1f}%")
        print(f"   📊 Precision change: {precision_change:+.1f}%")
    
    # Save detailed results
    results_file = "/content/drive/MyDrive/superlakes/sam2_results/sam2_small_lake_tuning_results.csv" if IN_COLAB else "/content/sam2_parameter_tuning_results.csv"
    df_recall.to_csv(results_file, index=False)
    print(f"\n📁 Detailed results saved (ranked by recall): {results_file}")
    
else:
    print("⏸️ No tuning results to analyze")
    best_config = None

In [ ]:
# Analyze and rank results
if TUNING_COMPLETE and not df_results.empty:
    print("📊 Analyzing parameter tuning results...\n")
    
    # Sort by IoU (primary metric)
    df_sorted = df_results.sort_values('iou', ascending=False)
    
    print("🏆 TOP 10 CONFIGURATIONS (ranked by IoU):")
    print("=" * 100)
    
    top_10 = df_sorted.head(10)
    for idx, (_, row) in enumerate(top_10.iterrows(), 1):
        print(f"Rank {idx:2d}: {row['config_name']}")
        print(f"         Points/side: {row['points_per_side']:2d}, IoU thresh: {row['pred_iou_thresh']:.2f}, ")
        print(f"         Min area: {row['min_mask_region_area']:4d}, Stability: {row['stability_score_thresh']:.2f}")
        print(f"         📊 IoU: {row['iou']:.3f}, Precision: {row['precision']:.3f}, Recall: {row['recall']:.3f}, F1: {row['f1']:.3f}")
        print(f"         🎯 Masks: {row['num_masks']:3d}, Predicted: {row['predicted_lake_pixels']:,} pixels")
        print()
    
    # Best configuration details
    best_config = df_sorted.iloc[0]
    print(f"🎯 OPTIMAL CONFIGURATION:")
    print(f"   Name: {best_config['config_name']}")
    print(f"   points_per_side: {best_config['points_per_side']}")
    print(f"   pred_iou_thresh: {best_config['pred_iou_thresh']}")
    print(f"   min_mask_region_area: {best_config['min_mask_region_area']}")
    print(f"   stability_score_thresh: {best_config['stability_score_thresh']}")
    print(f"   📈 Performance: IoU={best_config['iou']:.3f}, F1={best_config['f1']:.3f}")
    
    # Save detailed results
    results_file = "/content/sam2_parameter_tuning_results.csv"
    df_sorted.to_csv(results_file, index=False)
    print(f"\n📁 Detailed results saved to: {results_file}")
    
    # Download link for Colab
    print(f"\n💾 To download results: files.download('{results_file}')")
    
else:
    print("⏸️ No tuning results to analyze")
    best_config = None

In [ ]:
# Create parameter effect visualizations
if TUNING_COMPLETE and not df_results.empty:
    print("📈 Creating parameter effect visualizations...")
    
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('SAM 2 Parameter Effects on Lake Detection Performance', fontsize=16, y=1.02)
    
    # 1. IoU vs points_per_side
    points_data = df_results.groupby('points_per_side')['iou'].agg(['max', 'mean', 'std']).reset_index()
    axes[0,0].bar(points_data['points_per_side'].astype(str), points_data['max'], 
                  alpha=0.7, color='skyblue', label='Best IoU')
    axes[0,0].scatter(points_data['points_per_side'].astype(str), points_data['mean'], 
                     color='red', s=50, label='Mean IoU', zorder=5)
    axes[0,0].set_title('IoU vs Points Per Side')
    axes[0,0].set_xlabel('Points Per Side')
    axes[0,0].set_ylabel('IoU Score')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. IoU vs pred_iou_thresh
    iou_data = df_results.groupby('pred_iou_thresh')['iou'].agg(['max', 'mean']).reset_index()
    axes[0,1].bar(iou_data['pred_iou_thresh'].astype(str), iou_data['max'], 
                  alpha=0.7, color='lightcoral', label='Best IoU')
    axes[0,1].scatter(iou_data['pred_iou_thresh'].astype(str), iou_data['mean'], 
                     color='darkred', s=50, label='Mean IoU', zorder=5)
    axes[0,1].set_title('IoU vs Prediction IoU Threshold')
    axes[0,1].set_xlabel('Prediction IoU Threshold')
    axes[0,1].set_ylabel('IoU Score')
    axes[0,1].legend()
    axes[0,1].tick_params(axis='x', rotation=45)
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. IoU vs min_mask_region_area
    area_data = df_results.groupby('min_mask_region_area')['iou'].agg(['max', 'mean']).reset_index()
    axes[0,2].bar(area_data['min_mask_region_area'].astype(str), area_data['max'], 
                  alpha=0.7, color='lightgreen', label='Best IoU')
    axes[0,2].scatter(area_data['min_mask_region_area'].astype(str), area_data['mean'], 
                     color='darkgreen', s=50, label='Mean IoU', zorder=5)
    axes[0,2].set_title('IoU vs Min Mask Region Area')
    axes[0,2].set_xlabel('Min Mask Region Area (pixels)')
    axes[0,2].set_ylabel('IoU Score')
    axes[0,2].legend()
    axes[0,2].tick_params(axis='x', rotation=45)
    axes[0,2].grid(True, alpha=0.3)
    
    # 4. Precision vs Recall scatter
    scatter = axes[1,0].scatter(df_results['recall'], df_results['precision'], 
                               c=df_results['iou'], cmap='viridis', s=80, alpha=0.7)
    axes[1,0].set_title('Precision vs Recall\n(colored by IoU)')
    axes[1,0].set_xlabel('Recall')
    axes[1,0].set_ylabel('Precision')
    axes[1,0].grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=axes[1,0], label='IoU')
    
    # 5. Number of masks vs IoU
    axes[1,1].scatter(df_results['num_masks'], df_results['iou'], 
                     c=df_results['f1'], cmap='plasma', s=80, alpha=0.7)
    axes[1,1].set_title('Number of Masks vs IoU\n(colored by F1 score)')
    axes[1,1].set_xlabel('Number of Masks Generated')
    axes[1,1].set_ylabel('IoU Score')
    axes[1,1].grid(True, alpha=0.3)
    
    # 6. Configuration ranking
    top_configs = df_results.nlargest(8, 'iou')
    y_pos = np.arange(len(top_configs))
    axes[1,2].barh(y_pos, top_configs['iou'], alpha=0.7, color='gold')
    axes[1,2].set_yticks(y_pos)
    axes[1,2].set_yticklabels([name[:15] + '...' if len(name) > 15 else name 
                              for name in top_configs['config_name']], fontsize=9)
    axes[1,2].set_title('Top 8 Configurations by IoU')
    axes[1,2].set_xlabel('IoU Score')
    axes[1,2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Save plot
    plot_file = "/content/sam2_parameter_analysis.png"
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"📁 Parameter analysis plot saved to: {plot_file}")
    
else:
    print("⏸️ Skipping visualization (no tuning results)")

## 🎉 Generate Final Result with Optimal Configuration

In [ ]:
# Generate final result with best configuration
if SAM2_LOADED:
    if best_config is not None:
        print(f"🎯 Generating final result with optimal configuration: {best_config['config_name']}")
        
        # Extract optimal parameters
        optimal_params = {
            'points_per_side': int(best_config['points_per_side']),
            'pred_iou_thresh': float(best_config['pred_iou_thresh']),
            'min_mask_region_area': int(best_config['min_mask_region_area']),
            'stability_score_thresh': float(best_config['stability_score_thresh'])
        }
        
        # Create optimal mask generator
        optimal_generator = SAM2AutomaticMaskGenerator(sam2_model, **optimal_params)
        config_name = best_config['config_name']
        
    else:
        print("🔄 Using default configuration (no tuning performed)")
        optimal_generator = SAM2AutomaticMaskGenerator(sam2_model)
        optimal_params = "Default SAM 2 configuration"
        config_name = "default"
    
    # Generate final masks
    print("   Generating final masks...")
    final_masks = optimal_generator.generate(image)
    print(f"✅ Final configuration generated {len(final_masks)} masks")
    
    FINAL_RESULT_READY = True
    
else:
    print("⏸️ Cannot generate final result (SAM 2 not loaded)")
    FINAL_RESULT_READY = False
    final_masks = []
    optimal_params = None
    config_name = "none"

## 🖼️ Comprehensive Final Visualization

In [ ]:
# Create comprehensive final visualization
if FINAL_RESULT_READY:
    if GROUND_TRUTH_LOADED:
        # With ground truth comparison
        print("🖼️ Creating comprehensive visualization with ground truth comparison...")
        
        # Combine final masks
        final_combined_mask = np.zeros_like(ground_truth, dtype=bool)
        for mask in final_masks:
            final_combined_mask |= mask['segmentation']
        
        # Calculate final metrics
        final_metrics = calculate_metrics(ground_truth, final_combined_mask)
        
        # Create comprehensive visualization
        fig = plt.figure(figsize=(24, 16))
        
        # Create custom grid layout
        gs = fig.add_gridspec(3, 4, height_ratios=[1, 1, 0.6], width_ratios=[1, 1, 1, 1])
        
        # Row 1: Input and Ground Truth
        ax1 = fig.add_subplot(gs[0, 0])
        ax1.imshow(image)
        ax1.set_title(f'Input: {IMAGE_TYPE} Image\nShape: {image.shape}')
        ax1.axis('off')
        
        ax2 = fig.add_subplot(gs[0, 1])
        ax2.imshow(image)
        ax2.imshow(ground_truth, alpha=0.6, cmap='Blues')
        ax2.set_title(f'Ground Truth Lakes\n{ground_truth.sum():,} pixels ({ground_truth.mean()*100:.1f}%)')
        ax2.axis('off')
        
        # Row 1: SAM 2 Results
        ax3 = fig.add_subplot(gs[0, 2])
        ax3.imshow(image)
        show_anns(final_masks)
        ax3.set_title(f'SAM 2 Individual Masks\n{len(final_masks)} masks detected')
        ax3.axis('off')
        
        ax4 = fig.add_subplot(gs[0, 3])
        ax4.imshow(image)
        ax4.imshow(final_combined_mask, alpha=0.6, cmap='Reds')
        ax4.set_title(f'SAM 2 Combined Result\n{final_combined_mask.sum():,} pixels ({final_combined_mask.mean()*100:.1f}%)')
        ax4.axis('off')
        
        # Row 2: Detailed Comparison
        ax5 = fig.add_subplot(gs[1, 0:2])
        # Create detailed comparison overlay
        comparison = np.zeros((*ground_truth.shape, 3), dtype=np.uint8)
        comparison[ground_truth & final_combined_mask] = [0, 255, 0]      # True positive (green)
        comparison[ground_truth & ~final_combined_mask] = [0, 0, 255]     # False negative (blue)
        comparison[~ground_truth & final_combined_mask] = [255, 0, 0]     # False positive (red)
        
        ax5.imshow(image)
        ax5.imshow(comparison, alpha=0.7)
        ax5.set_title('Detailed Pixel-Level Comparison\n🟢 Correct Detection  🔵 Missed Lakes  🔴 False Positives')
        ax5.axis('off')
        
        # Row 2: Side-by-side comparison
        ax6 = fig.add_subplot(gs[1, 2:4])
        # Create side-by-side overlay
        overlay_image = image.copy()
        # Ground truth on left half (blue)
        left_half = slice(None), slice(None, image.shape[1]//2)
        overlay_blue = np.zeros((*ground_truth.shape, 4))
        overlay_blue[ground_truth] = [0, 0, 1, 0.6]
        
        # SAM 2 result on right half (red)
        right_half = slice(None), slice(image.shape[1]//2, None)
        overlay_red = np.zeros((*ground_truth.shape, 4))
        overlay_red[final_combined_mask] = [1, 0, 0, 0.6]
        
        ax6.imshow(image)
        ax6.imshow(overlay_blue[left_half], alpha=0.6, extent=[0, image.shape[1]//2, image.shape[0], 0])
        ax6.imshow(overlay_red[right_half], alpha=0.6, extent=[image.shape[1]//2, image.shape[1], image.shape[0], 0])
        ax6.axvline(x=image.shape[1]//2, color='white', linewidth=3, linestyle='--')
        ax6.set_title('Side-by-Side Comparison\n🔵 Ground Truth (left)  🔴 SAM 2 Result (right)')
        ax6.axis('off')
        
        # Row 3: Metrics and Configuration
        ax7 = fig.add_subplot(gs[2, :])
        ax7.axis('off')
        
        # Create metrics text
        metrics_text = f"""
🎯 OPTIMAL CONFIGURATION: {config_name}
📊 PERFORMANCE METRICS:
   • IoU (Intersection over Union): {final_metrics['iou']:.3f}
   • Precision: {final_metrics['precision']:.3f}
   • Recall: {final_metrics['recall']:.3f} 
   • F1 Score: {final_metrics['f1']:.3f}

🔧 SAM 2 PARAMETERS:
   • Points per side: {optimal_params['points_per_side']}
   • Prediction IoU threshold: {optimal_params['pred_iou_thresh']}
   • Min mask region area: {optimal_params['min_mask_region_area']} pixels
   • Stability score threshold: {optimal_params['stability_score_thresh']}

📈 DETECTION SUMMARY:
   • Ground truth lake pixels: {ground_truth.sum():,}
   • SAM 2 detected pixels: {final_combined_mask.sum():,}
   • Correctly identified: {(ground_truth & final_combined_mask).sum():,} pixels
   • Missed lakes: {(ground_truth & ~final_combined_mask).sum():,} pixels
   • False positives: {(~ground_truth & final_combined_mask).sum():,} pixels
"""
        
        ax7.text(0.05, 0.95, metrics_text, transform=ax7.transAxes, fontsize=12, 
                verticalalignment='top', fontfamily='monospace',
                bbox=dict(boxstyle='round,pad=0.5', facecolor='lightgray', alpha=0.8))
        
        plt.tight_layout()
        
        # Save comprehensive result
        final_plot_file = f"/content/sam2_final_result_comprehensive_{config_name}.png"
        plt.savefig(final_plot_file, dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f"\n🎉 FINAL RESULTS SUMMARY:")
        print(f"   🏆 Best configuration: {config_name}")
        print(f"   📊 IoU: {final_metrics['iou']:.3f}")
        print(f"   📊 Precision: {final_metrics['precision']:.3f}")
        print(f"   📊 Recall: {final_metrics['recall']:.3f}")
        print(f"   📊 F1 Score: {final_metrics['f1']:.3f}")
        print(f"   📁 Comprehensive visualization saved to: {final_plot_file}")
        
    else:
        # Without ground truth - just show result
        print("🖼️ Creating visualization without ground truth comparison...")
        
        fig, axes = plt.subplots(1, 2, figsize=(20, 10))
        
        # Original image
        axes[0].imshow(image)
        axes[0].set_title(f'Original {IMAGE_TYPE} Image')
        axes[0].axis('off')
        
        # SAM 2 result
        axes[1].imshow(image)
        show_anns(final_masks)
        axes[1].set_title(f'SAM 2 Lake Detection Result\n{len(final_masks)} masks detected')
        axes[1].axis('off')
        
        plt.tight_layout()
        
        final_plot_file = f"/content/sam2_final_result_{config_name}.png"
        plt.savefig(final_plot_file, dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f"📁 Final result saved to: {final_plot_file}")
        
else:
    print("⏸️ Skipping final visualization (no final result available)")

## 💾 Export Results and Configuration

In [ ]:
# Export optimal configuration and code template
if best_config is not None:
    print("💾 Exporting optimal configuration and usage code...")
    
    # Create results directory in Google Drive
    if IN_COLAB:
        results_dir = os.path.join(DRIVE_DATA_PATH, "sam2_results")
    else:
        results_dir = "/content/sam2_results"
    
    os.makedirs(results_dir, exist_ok=True)
    print(f"📁 Results directory: {results_dir}")
    
    # Create optimal configuration dictionary
    optimal_config = {
        'configuration_name': best_config['config_name'],
        'sam2_parameters': {
            'points_per_side': int(best_config['points_per_side']),
            'pred_iou_thresh': float(best_config['pred_iou_thresh']),
            'min_mask_region_area': int(best_config['min_mask_region_area']),
            'stability_score_thresh': float(best_config['stability_score_thresh'])
        },
        'performance_metrics': {
            'iou': float(best_config['iou']),
            'precision': float(best_config['precision']),
            'recall': float(best_config['recall']),
            'f1_score': float(best_config['f1'])
        },
        'test_conditions': {
            'image_type': IMAGE_TYPE,
            'image_shape': image.shape if IMAGE_LOADED else None,
            'ground_truth_lake_pixels': int(ground_truth.sum()) if GROUND_TRUTH_LOADED else None,
            'device_used': str(device),
            'test_image': IMAGE_FILES[IMAGE_TYPE],
            'ground_truth_file': GROUND_TRUTH_FILE
        },
        'usage_notes': [
            "This configuration was optimized for glacial lake detection",
            f"Tested on {IMAGE_TYPE} satellite imagery",
            "Performance may vary on different image types or regions",
            "Consider re-tuning for significantly different lake sizes or imagery"
        ]
    }
    
    # Save configuration as JSON to Drive
    config_file = os.path.join(results_dir, "optimal_sam2_lake_detection_config.json")
    with open(config_file, 'w') as f:
        json.dump(optimal_config, f, indent=2)
    
    print(f"✅ Optimal configuration saved to Google Drive: {config_file}")
    
    # Create usage code template
    code_template = f'''
# ============================================================================
# OPTIMAL SAM 2 CONFIGURATION FOR LAKE DETECTION
# Generated from parameter tuning on {IMAGE_TYPE} imagery
# Performance: IoU={best_config['iou']:.3f}, F1={best_config['f1']:.3f}
# ============================================================================

from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
import numpy as np

# Load SAM 2 model
sam2_model = build_sam2(
    "configs/sam2.1/sam2.1_hiera_l.yaml",
    "checkpoints/sam2.1_hiera_large.pt", 
    device="cuda"  # or "cpu"
)

# Create optimal mask generator for lake detection
optimal_lake_detector = SAM2AutomaticMaskGenerator(
    sam2_model,
    points_per_side={best_config['points_per_side']},
    pred_iou_thresh={best_config['pred_iou_thresh']},
    min_mask_region_area={best_config['min_mask_region_area']},
    stability_score_thresh={best_config['stability_score_thresh']}
)

# Apply to new image (ensure image is in correct format)
# For {IMAGE_TYPE} images: load and prepare as done in this notebook
new_masks = optimal_lake_detector.generate(new_image)

# Combine masks into binary lake detection
lake_mask = np.zeros((new_image.shape[0], new_image.shape[1]), dtype=bool)
for mask in new_masks:
    lake_mask |= mask['segmentation']

# Expected performance on similar imagery:
# - IoU: {best_config['iou']:.3f}
# - Precision: {best_config['precision']:.3f} 
# - Recall: {best_config['recall']:.3f}
# - F1 Score: {best_config['f1']:.3f}

print(f"Detected {{len(new_masks)}} lake segments")
print(f"Total lake pixels: {{lake_mask.sum():,}}")
'''
    
    # Save code template to Drive
    code_file = os.path.join(results_dir, "optimal_sam2_lake_detection_code.py")
    with open(code_file, 'w') as f:
        f.write(code_template)
    
    print(f"✅ Usage code template saved to Google Drive: {code_file}")
    
    # Save detailed results CSV to Drive
    if 'df_results' in locals() and not df_results.empty:
        results_csv = os.path.join(results_dir, "sam2_parameter_tuning_results.csv")
        df_results.to_csv(results_csv, index=False)
        print(f"✅ Detailed results CSV saved to Google Drive: {results_csv}")
    
    # Save visualizations to Drive
    if 'final_plot_file' in locals():
        # Copy final plot to Drive
        drive_plot_file = os.path.join(results_dir, f"sam2_final_result_comprehensive_{best_config['config_name']}.png")
        if os.path.exists(final_plot_file):
            import shutil
            shutil.copy2(final_plot_file, drive_plot_file)
            print(f"✅ Final visualization saved to Google Drive: {drive_plot_file}")
    
    # Display the template
    print("\n🚀 READY-TO-USE CODE FOR NEW IMAGES:")
    print("=" * 80)
    print(code_template)
    
    # Summary of saved files
    print(f"\n📁 ALL RESULTS SAVED TO GOOGLE DRIVE:")
    print(f"   📂 Results folder: {results_dir}")
    print(f"   📄 Configuration JSON: optimal_sam2_lake_detection_config.json")
    print(f"   🐍 Python code: optimal_sam2_lake_detection_code.py")
    if 'df_results' in locals() and not df_results.empty:
        print(f"   📊 Detailed results: sam2_parameter_tuning_results.csv")
    if 'final_plot_file' in locals():
        print(f"   🖼️ Final visualization: sam2_final_result_comprehensive_{best_config['config_name']}.png")
    
    print(f"\n💡 Access these files anytime from your Google Drive at: {results_dir}")
    
else:
    print("⏸️ No optimal configuration to export (parameter tuning not completed)")
    
    # Still provide default usage template
    print("\n📝 DEFAULT SAM 2 USAGE FOR LAKE DETECTION:")
    default_template = '''
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator

# Load SAM 2
sam2_model = build_sam2("configs/sam2.1/sam2.1_hiera_l.yaml", 
                        "checkpoints/sam2.1_hiera_large.pt", device="cuda")

# Create mask generator with default parameters
mask_generator = SAM2AutomaticMaskGenerator(sam2_model)

# Generate masks
masks = mask_generator.generate(your_image)
print(f"Generated {len(masks)} masks")
'''
    print(default_template)

## 🎉 Analysis Complete!

This notebook has systematically tested SAM 2 parameters for glacial lake detection. 

### What was accomplished:
1. ✅ **Environment Setup**: Configured Google Colab with SAM 2 and dependencies
2. ✅ **Data Loading**: Loaded satellite imagery and ground truth lake masks
3. ✅ **Parameter Tuning**: Tested multiple SAM 2 configurations systematically
4. ✅ **Performance Analysis**: Evaluated each configuration using IoU, precision, recall, F1
5. ✅ **Optimal Configuration**: Identified best parameters for your specific use case
6. ✅ **Results Export**: Generated ready-to-use code and configuration files

### Next steps:
- **Download the results** using the provided commands
- **Apply the optimal configuration** to your full dataset
- **Validate on additional images** to confirm performance
- **Consider re-tuning** if working with significantly different imagery

### Need help?
- Check the exported configuration JSON for complete parameter details
- Use the provided code template for applying to new images
- Consider the performance metrics when evaluating results on new data

**Happy lake detecting! 🏔️💧**